In [1]:
from geopy.geocoders import Nominatim
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point
import openrouteservice
import geocoder

In [2]:
df = pd.read_csv('C:/Users/mobas/OneDrive/Desktop/EVAT.charging_stations.csv')

In [3]:
df

,_id,cost,charging_points,pay_at_location,membership_required,access_key_required,is_operational,latitude,longitude,operator,connection_type,current_type,charging_points_flag,power_output
0,66d7e0a1cdf87e8b5d63d809,Unknown,1,Unknown,Unknown,Unknown,Unknown,-31.936252,115.871531,Unknown,Unknown,Unknown,1,NaN
1,66d7e0a1cdf87e8b5d63d80a,Unknown,1,Unknown,Unknown,Unknown,Unknown,-32.935880,151.643848,ChargePoint,Type 1 (J1772),Unknown,1,NaN
2,66d7e0a1cdf87e8b5d63d80b,Unknown,2,Unknown,Unknown,Unknown,Unknown,-41.218297,146.412523,Unknown,Type 2 (Socket Only),Unknown,0,NaN
3,66d7e0a1cdf87e8b5d63d80c,Unknown,8,Unknown,Unknown,Unknown,Unknown,-34.755480,149.720994,Tesla Supercharger,Unknown,Unknown,0,NaN
4,66d7e0a1cdf87e8b5d63d80d,Unknown,4,Unknown,Unknown,Unknown,Unknown,-37.887228,145.082491,"Tesla, Inc.",Unknown,Unknown,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1626,66d7e0a1cdf87e8b5d63de63,Unknown,1,Unknown,Unknown,Unknown,Unknown,-33.693647,151.018619,Blink Charging,Type 1 (J1772),Unknown,1,NaN
1627,66d7e0a1cdf87e8b5d63de64,Unknown,1,false,true,true,true,-31.937153,115.840552,Chargestar (AU),Type 1 (J1772),AC (Single-Phase),1,7.7
1628,66d7e0a1cdf87e8b5d63de65,Unknown,1,Unknown,Unknown,Unknown,Unknown,-35.240510,149.087110,ChargePoint,NEMA 5-20R,Unknown,1,NaN
1629,66d7e0a1cdf87e8b5d63de66,Unknown,1,Unknown,Unknown,Unknown,Unknown,-33.883580,151.191530,ChargePoint,NEMA 5-20R,Unknown,1,NaN


In [4]:
df = df[['cost', 'charging_points', 'pay_at_location', 'is_operational', 'latitude','longitude','operator', 'connection_type']]

In [5]:
df

,cost,charging_points,pay_at_location,is_operational,latitude,longitude,operator,connection_type
0,Unknown,1,Unknown,Unknown,-31.936252,115.871531,Unknown,Unknown
1,Unknown,1,Unknown,Unknown,-32.935880,151.643848,ChargePoint,Type 1 (J1772)
2,Unknown,2,Unknown,Unknown,-41.218297,146.412523,Unknown,Type 2 (Socket Only)
3,Unknown,8,Unknown,Unknown,-34.755480,149.720994,Tesla Supercharger,Unknown
4,Unknown,4,Unknown,Unknown,-37.887228,145.082491,"Tesla, Inc.",Unknown
...,...,...,...,...,...,...,...,...
1626,Unknown,1,Unknown,Unknown,-33.693647,151.018619,Blink Charging,Type 1 (J1772)
1627,Unknown,1,false,true,-31.937153,115.840552,Chargestar (AU),Type 1 (J1772)
1628,Unknown,1,Unknown,Unknown,-35.240510,149.087110,ChargePoint,NEMA 5-20R
1629,Unknown,1,Unknown,Unknown,-33.883580,151.191530,ChargePoint,NEMA 5-20R


In [6]:
geolocator = Nominatim(user_agent="skywalker1254")

In [7]:
def reverse_geocode(lat, lon):
    for attempt in range(3):  # Retry up to 3 times
        try:
            location = geolocator.reverse((lat, lon), exactly_one=True)
            return location.address if location else "Address not found"
        except GeopyError as e:
            print(f"Attempt {attempt + 1}: {e}")
            time.sleep(2)  
    return "Error: Unable to fetch address"

In [8]:
subset_df = df[['cost', 'charging_points', 'pay_at_location', 'is_operational', 'latitude','longitude','operator', 'connection_type']].copy()
subset_df['Address'] = subset_df.apply(lambda row: reverse_geocode(row['latitude'], row['longitude']), axis=1)

In [9]:
subset_df

,cost,charging_points,pay_at_location,is_operational,latitude,longitude,operator,connection_type,Address
0,Unknown,1,Unknown,Unknown,-31.936252,115.871531,Unknown,Unknown,"Kaata Lane, Mount Lawley, City of Vincent, Wes..."
1,Unknown,1,Unknown,Unknown,-32.935880,151.643848,ChargePoint,Type 1 (J1772),"ChargePoint, Montgomery Street, Argenton, Newc..."
2,Unknown,2,Unknown,Unknown,-41.218297,146.412523,Unknown,Type 2 (Socket Only),"Bass Highway, Latrobe, Tasmania, 7307, Australia"
3,Unknown,8,Unknown,Unknown,-34.755480,149.720994,Tesla Supercharger,Unknown,"Tesla Supercharger, 201, Sloane Street, Goulbu..."
4,Unknown,4,Unknown,Unknown,-37.887228,145.082491,"Tesla, Inc.",Unknown,"Tesla Destination Charger, 1341, Dandenong Roa..."
...,...,...,...,...,...,...,...,...,...
1626,Unknown,1,Unknown,Unknown,-33.693647,151.018619,Blink Charging,Type 1 (J1772),"3, Allen Way, Round Corner, Dural, Sydney, The..."
1627,Unknown,1,false,true,-31.937153,115.840552,Chargestar (AU),Type 1 (J1772),"Mitchell Freeway, Leederville, City of Vincent..."
1628,Unknown,1,Unknown,Unknown,-35.240510,149.087110,ChargePoint,NEMA 5-20R,"University of Canberra, University Drive, Bruc..."
1629,Unknown,1,Unknown,Unknown,-33.883580,151.191530,ChargePoint,NEMA 5-20R,"Derby Place, Glebe, Inner West, Sydney, Counci..."


In [22]:
subset_df.to_csv("C:/Users/mobas/OneDrive/Desktop/new.csv", index= False)

In [24]:
df1 = pd.read_csv('C:/Users/mobas/OneDrive/Desktop/iMAGE/new.csv')

In [26]:
df1

,cost,charging_points,pay_at_location,is_operational,latitude,longitude,operator,connection_type,Address
0,Unknown,1,Unknown,Unknown,-31.936252,115.871531,Unknown,Unknown,"Kaata Lane, Mount Lawley, City of Vincent, Wes..."
1,Unknown,1,Unknown,Unknown,-32.935880,151.643848,ChargePoint,Type 1 (J1772),"ChargePoint, Montgomery Street, Argenton, Newc..."
2,Unknown,2,Unknown,Unknown,-41.218297,146.412523,Unknown,Type 2 (Socket Only),"Bass Highway, Latrobe, Tasmania, 7307, Australia"
3,Unknown,8,Unknown,Unknown,-34.755480,149.720994,Tesla Supercharger,Unknown,"Tesla Supercharger, 201, Sloane Street, Goulbu..."
4,Unknown,4,Unknown,Unknown,-37.887228,145.082491,"Tesla, Inc.",Unknown,"Tesla Destination Charger, 1341, Dandenong Roa..."
...,...,...,...,...,...,...,...,...,...
1626,Unknown,1,Unknown,Unknown,-33.693647,151.018619,Blink Charging,Type 1 (J1772),"3, Allen Way, Round Corner, Dural, Sydney, The..."
1627,Unknown,1,false,true,-31.937153,115.840552,Chargestar (AU),Type 1 (J1772),"Leederville Parade, Leederville, City of Vince..."
1628,Unknown,1,Unknown,Unknown,-35.240510,149.087110,ChargePoint,NEMA 5-20R,"University of Canberra, University Drive, Bruc..."
1629,Unknown,1,Unknown,Unknown,-33.883580,151.191530,ChargePoint,NEMA 5-20R,"Derby Place, Glebe, Inner West, Sydney, Counci..."


In [28]:
map = df1[['charging_points', 'operator', 'connection_type', 'Address','latitude', 'longitude']]

In [30]:
map

,charging_points,operator,connection_type,Address,latitude,longitude
0,1,Unknown,Unknown,"Kaata Lane, Mount Lawley, City of Vincent, Wes...",-31.936252,115.871531
1,1,ChargePoint,Type 1 (J1772),"ChargePoint, Montgomery Street, Argenton, Newc...",-32.935880,151.643848
2,2,Unknown,Type 2 (Socket Only),"Bass Highway, Latrobe, Tasmania, 7307, Australia",-41.218297,146.412523
3,8,Tesla Supercharger,Unknown,"Tesla Supercharger, 201, Sloane Street, Goulbu...",-34.755480,149.720994
4,4,"Tesla, Inc.",Unknown,"Tesla Destination Charger, 1341, Dandenong Roa...",-37.887228,145.082491
...,...,...,...,...,...,...
1626,1,Blink Charging,Type 1 (J1772),"3, Allen Way, Round Corner, Dural, Sydney, The...",-33.693647,151.018619
1627,1,Chargestar (AU),Type 1 (J1772),"Leederville Parade, Leederville, City of Vince...",-31.937153,115.840552
1628,1,ChargePoint,NEMA 5-20R,"University of Canberra, University Drive, Bruc...",-35.240510,149.087110
1629,1,ChargePoint,NEMA 5-20R,"Derby Place, Glebe, Inner West, Sydney, Counci...",-33.883580,151.191530


In [32]:
list= map[['Address','charging_points', 'operator', 'connection_type', 'latitude', 'longitude']].values.tolist()

In [34]:
list

[['Kaata Lane, Mount Lawley, City of Vincent, Western Australia, 6050, Australia',
  1,
  'Unknown',
  'Unknown',
  -31.9362523,
  115.8715309],
 ['ChargePoint, Montgomery Street, Argenton, Newcastle-Maitland, City of Lake Macquarie, New South Wales, 2284, Australia',
  1,
  'ChargePoint',
  'Type 1 (J1772)',
  -32.9358797,
  151.6438477],
 ['Bass Highway, Latrobe, Tasmania, 7307, Australia',
  2,
  'Unknown',
  'Type 2 (Socket Only)',
  -41.218297,
  146.4125226],
 ['Tesla Supercharger, 201, Sloane Street, Goulburn, Goulburn Mulwaree Council, New South Wales, 2580, Australia',
  8,
  'Tesla Supercharger',
  'Unknown',
  -34.7554799,
  149.7209942],
 ['Tesla Destination Charger, 1341, Dandenong Road, Malvern East, Melbourne, City of Stonnington, Victoria, 3145, Australia',
  4,
  'Tesla, Inc.',
  'Unknown',
  -37.8872284,
  145.0824905],
 ['Quality Drive, Dandenong South, Melbourne, City of Greater Dandenong, Victoria, 3175, Australia',
  2,
  'Unknown',
  'Unknown',
  -38.0270897,
  1

In [36]:
aus_map = folium.Map(location=[-41.23530922,146.4083866])
fg=folium.FeatureGroup(name='Ev_map')
for i in list:
    fg.add_child(folium.Marker(location=[i[4],i[5]], popup=[i[0],i[1],i[2],i[3]], icon=folium.Icon(color='green')))
aus_map.add_child(fg)

In [38]:
aus_map.save('C:/Users/mobas/OneDrive/Desktop/Aus_map.html')

In [40]:
current_location = geocoder.ip('me')

if current_location.ok:
    current_coords = (current_location.latlng[0], current_location.latlng[1])
    print(f"Current Location: {current_coords}")
else:
    print("Unable to fetch current location.")
    current_coords = (0, 0)  


Current Location: (-37.814, 144.9633)


In [42]:
charger_gdf = gpd.GeoDataFrame(
    map,
    geometry=gpd.points_from_xy(map.longitude, map.latitude),
    crs="EPSG:3857"
)


In [44]:
current_point = Point(current_coords[1], current_coords[0])  # Point(lon, lat)
charger_gdf['distance'] = charger_gdf.geometry.distance(current_point)
nearest_charger = charger_gdf.loc[charger_gdf['distance'].idxmin()]
print(f"Nearest Charger: {nearest_charger['Address']} at {nearest_charger.geometry}")

Nearest Charger: Hardware Lane, Melbourne, City of Melbourne, Victoria, 3000, Australia at POINT (144.9618065 -37.8143461)


In [46]:
api_key = "5b3ce3597851110001cf62483ba95147bc4d4f9ba6e44d04127c4d06"  
client = openrouteservice.Client(key=api_key)

In [48]:
coords = [(current_coords[1], current_coords[0]), 
          (nearest_charger.geometry.x, nearest_charger.geometry.y)]

In [50]:
route = client.directions(coords, profile='driving-car', format='geojson')
m = folium.Map(location=current_coords, zoom_start=12)

In [52]:
folium.Marker(
    location=current_coords,
    popup="Current Location",
    icon=folium.Icon(color='blue')
).add_to(m)
folium.Marker(
    location=(nearest_charger.geometry.y, nearest_charger.geometry.x),
    popup=nearest_charger['Address'],
    icon=folium.Icon(color='green')
).add_to(m)

In [54]:
folium.GeoJson(route, name='Route').add_to(m)
m